In [1]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from datetime import datetime, timedelta
import pandas as pd
from urllib.request import urlopen
import json

In [2]:
def nameLookup(itemID):
    itemURL = "https://cafemaker.wakingsands.com/item/" + str(itemID)
    itemInfoRaw = urlopen(itemURL).read().decode('utf-8')
    # print(itemInfoRaw)
    itemInfo = json.loads(itemInfoRaw)
    return itemInfo['Name']

In [3]:
# query = { 'itemID': 36258 }

In [4]:
one_day_ago = datetime.utcnow() - timedelta(hours=24)
one_week_ago = datetime.utcnow() - timedelta(days=7)

In [5]:
pipeline = [
    {
        "$match": {
            "lastUploadTime": {"$gte": one_day_ago}
        }
    },
    {
        "$group": {
            "_id": "$itemID",
            "viewCount": {"$sum": 1}
        }
    },
    {
        "$sort": {"viewCount": -1}
    },
    {
        "$limit": 10
    },
    {
        "$project": {
            "itemID": "$_id",
            "viewCount": "$viewCount",
            "_id": 0
        }
    }
]

In [6]:
uri = 'put uri in file mongo_uri'
with open('mongo_uri', 'r') as reader:
    uri = reader.read()
client = MongoClient(uri, server_api=ServerApi('1'))
db = client["universalis"]
collection = db["ws"]

# cursor = collection.find(query, {"_id": 0})
# for i in cursor:
#     print(i)

result = list(collection.aggregate(pipeline))
# print(result)

In [7]:
df = pd.DataFrame(result)
name = []
for i in df['itemID']:
    n = nameLookup(i)
    # print(n)
    name.append(n)
df['name'] = name

In [8]:
df[['itemID', 'name', 'viewCount']]

,itemID,name,viewCount
0,39591,陈旧的蛇牛革地图,212
1,36612,陈旧的金毗罗鳄革地图,167
2,17836,陈旧的瞪羚革地图,90
3,6692,陈旧的毒蜥蜴革地图,85
4,39872,烧烤暗色茄子,68
5,39706,软银矿,61
6,38936,地鸣灵砂,52
7,39709,油菜花,49
8,36260,彩蝶鳞粉,48
9,40700,轻柔棉布,46
